In [ ]:
import os
import glob
import math
import random
import gc

import numpy as np
import PIL
from PIL import Image
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
from tensorflow.python.keras import regularizers
from tensorflow.python.keras import backend as K
from tensorflow.python.keras.backend import set_session
from tensorflow.python.keras.models import Model, Sequential, load_model
from tensorflow.python.keras.layers import Conv2D, Dense, Input, MaxPooling2D, UpSampling2D, Lambda, Dropout, Concatenate, Add
from tensorflow.python.keras.layers import Conv2DTranspose, Activation, Cropping2D, BatchNormalization
from tensorflow.python.keras.preprocessing.image import load_img, img_to_array, array_to_img
from tensorflow.python.keras.callbacks import TensorBoard, ModelCheckpoint
from tensorflow.python.keras.optimizers import Adam, SGD

config = tf.ConfigProto(  
    gpu_options=tf.GPUOptions(
        visible_device_list="0", # specify GPU number
        allow_growth=True
    )
)
sess = tf.Session(config=config)


In [ ]:
NO_CLASS = 20
IGNORE_LABEL = np.uint8(19)
RESIZE_HEIGHT =512 #image height for network inputting
RESIZE_WIDTH = 1024  #image width for network inputting
#KNL_INITLZ = 'he_uniform'
KNL_INITLZ = 'he_normal'

#CRP_TYPE = 0 # pixel value = 0
CRP_TYPE = 1 # pixel value = salt_and_pepper noise
PRB_PXLCRP = 0.5 #Coruption Type: bengio, probability of each pixel corruption

LEARN_RATE = np.float32(1.0e-4)
BETA_1 = np.float32(0.99) # for adam
MOMENTUM= np.float32(0.9)

L2_VALUE1 = np.float32(1.0e-3)
L2_VALUE2 = np.float32(1.0e-3)
L2_VALUE3 = np.float32(1.0e-3)
L2_VALUE4 = np.float32(1.0e-3)

FILT_SIZE = (3, 3)
FILT_NO1 = 96
FILT_NO2 = 64
FILT_NO3 = 32
FILT_NO4 = 16

BATCH_SIZE = 4
EPOCHS1 = 1000
EPOCHS2 = 500

#PERIOD = 10


In [ ]:
path_train_image = "C:/Users/test/Documents/Cityscapes/Image/Train_Image"  #path to train image folder
path_val_image = "C:/Users/test/Documents/Cityscapes/Image/Val_Image"  #path to val image folder

checkpoint_path = "C:/Users/test/Documents/Cityscapes/Weights/best_weights.hdf5"  #path to checkpoint(Weights) folder

encoder_path = "C:/Users/test/Documents/Cityscapes/Encoder_Weights/encoder.hdf5"  #path to Encoder_Weights folder

path_History = "C:/Users/test/Documents/Cityscapes/History"  #path to History folder

In [ ]:
def data_transform1(path_image):
    
    no_image = 0
    input_image_array_list = []
    filename = os.listdir(path_image)[0]
    image = Image.open(path_image + "/" + filename)
    width = image.width
    tg_size = (RESIZE_WIDTH, RESIZE_HEIGHT)
    for filename in os.listdir(path_image):
        image = Image.open(path_image + "/" + filename)
        image = image.convert("RGB")
        if width != RESIZE_WIDTH:
            image = image.resize(tg_size, PIL.Image.ANTIALIAS)
        image_array = np.array(image, dtype=np.uint8)
        
#        no_ptbpxl = 0
        if CRP_TYPE == 0:
            for y in range(RESIZE_HEIGHT):
                for x in range(RESIZE_WIDTH):
                    randu = random.uniform(0.0, 1.0)
                    if randu <= PRB_PXLCRP:
#                        no_ptbpxl += 1
                        image_array.itemset((y, x, 0), 0)
                        image_array.itemset((y, x, 1), 0)
                        image_array.itemset((y, x, 2), 0)
        if CRP_TYPE == 1:
            for y in range(RESIZE_HEIGHT):
                for x in range(RESIZE_WIDTH):
                    randu1 = random.uniform(0.0, 1.0)
                    if randu1 <= PRB_PXLCRP:
#                        no_ptbpxl += 1
                        randu2 = random.uniform(0.0, 1.0)
                        if randu2 <= 0.5:
                            image_array.itemset((y, x, 0), 0)
                            image_array.itemset((y, x, 1), 0)
                            image_array.itemset((y, x, 2), 0)
                        else:
                            image_array.itemset((y, x, 0), 255)
                            image_array.itemset((y, x, 1), 255)
                            image_array.itemset((y, x, 2), 255)
                                
        input_image_array_list.append(image_array)
#        print("no_ptbpxl = ", no_ptbpxl)
        if no_image%100 == 0:
            print("no_image = ", no_image)
        no_image += 1
#        if no_image == 20:  #break for debug mode
#            break
            
    return no_image, input_image_array_list

In [ ]:
crpt_no_image, crpt_image_array_list = data_transform1(path_train_image)
print("crpt_no_image = ", crpt_no_image)
print("crpt data have been prepared successfully")


In [ ]:
def data_transform2(path_image):
    no_image = 0
    image_array_list = []
    filename = os.listdir(path_image)[0]
    image = Image.open(path_image + "/" + filename)
    width = image.width
    tg_size = (RESIZE_WIDTH, RESIZE_HEIGHT)
    for filename in os.listdir(path_image):
        no_image += 1
        image = Image.open(path_image + "/" + filename)
        image = image.convert("RGB")
        if width != RESIZE_WIDTH:
            image = image.resize(tg_size, PIL.Image.ANTIALIAS)    
        image_array = np.asarray(image, dtype=np.uint8)
        image_array_list.append(image_array)
            
#        if no_image == 20:  #break for debug mode
#            break
            
    return no_image, image_array_list

In [ ]:
orgl_no_image, orgl_image_array_list = data_transform2(path_train_image)
print("orgl_no_image = ", orgl_no_image)
print("orgl data have been prepared successfully")


In [ ]:
val_no_image, val_image_array_list = data_transform2(path_val_image)
print("val_no_image = ", val_no_image)
print("val data have been prepared successfully")


In [ ]:
print("display processed original image for checking")

from IPython.display import display_png

no = 0
for i in range(2):
    print("processed no = ", no)
    orgl_image = Image.fromarray(np.uint8(orgl_image_array_list[i]), mode="RGB")
    display_png(orgl_image)
    no += 1
    
print("final processed no = ", no)

In [ ]:
print("display processed corrupted image for checking")

from IPython.display import display_png

no = 0
for i in range(2):
    print("processed no = ", no)
    crpt_image = Image.fromarray(np.uint8(crpt_image_array_list[i]), mode="RGB")
    display_png(crpt_image)
    no += 1
    
print("final processed no = ", no)

In [ ]:
print("display processed val data for checking")

from IPython.display import display_png

no = 0
for i in range(2):
    print("processed no = ", no)
    val_image = Image.fromarray(np.uint8(val_image_array_list[i]), mode="RGB")
    display_png(val_image)
    no += 1
    
print("final processed no = ", no)

In [ ]:
orgl_data = np.array(orgl_image_array_list, dtype=np.float32)
orgl_data = orgl_data / np.float32(255.0)
orgl_data = orgl_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 3))

print("orgl_data.shape = ", orgl_data.shape)
print("orgl_data have been transformed successfully")

In [ ]:
crpt_data = np.array(crpt_image_array_list, dtype=np.float32)
crpt_data = crpt_data / np.float32(255.0)
crpt_data = crpt_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 3))

print("crpt_data.shape = ", crpt_data.shape)
print("crpt_data have been transformed successfully")

In [ ]:
val_input_data = np.array(val_image_array_list, dtype=np.float32)
val_input_data = val_input_data / np.float32(255.0)
val_input_data = val_input_data.reshape((-1, RESIZE_HEIGHT, RESIZE_WIDTH, 3))

print("val_input_data.shape = ", val_input_data.shape)
print("val_input_data have been transformed successfully")

In [ ]:
def mtanh(x):
    return 0.5*tf.keras.activations.tanh(x) + 0.5

input = Input(shape=(RESIZE_HEIGHT, RESIZE_WIDTH, 3))

# Encoder
# Filter 1
e = Conv2D(FILT_NO1, FILT_SIZE, 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(input)
e = BatchNormalization()(e)
    
# Filter 2
e = Conv2D(FILT_NO2, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE2))(e)
e = BatchNormalization()(e)

# Filter 3
e = Conv2D(FILT_NO3, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE3))(e)
e = BatchNormalization()(e)

# Filter 4, output is bottleneck
e = Conv2D(FILT_NO4, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE4))(e)
bottleneck = BatchNormalization()(e)

# Decoder
# Filter 5
d = Conv2D(FILT_NO3, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE4))(bottleneck)
d = BatchNormalization()(d)

# Filter 6
d = Conv2D(FILT_NO2, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE3))(d)
d = BatchNormalization()(d)

# Filter 7
d = Conv2D(FILT_NO1, (3, 3), 1, activation='relu', padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE2))(d)
d = BatchNormalization()(d)

# Filter 8
decoded = Conv2D(3, (3, 3), 1, activation=mtanh, padding='same', kernel_initializer=KNL_INITLZ,
        kernel_regularizer=regularizers.l2(L2_VALUE1))(d)

model = Model(inputs=[input], outputs=[decoded])


In [ ]:
sgd = SGD(lr=LEARN_RATE, momentum=MOMENTUM, nesterov=True)
#adam = Adam(lr=LEARN_RATE, beta_1=BETA_1)

model.compile(
#    optimizer = adam,
    optimizer = sgd,
    loss='binary_crossentropy'
)


In [ ]:
model.summary()

In [ ]:
tsb=TensorBoard(log_dir='.logs')
modelcp = ModelCheckpoint(filepath = checkpoint_path,
                                  monitor='val_loss',
                                  verbose=1,
                                  save_best_only=True,
                                  save_weights_only=True,
                                  mode='min',
                                  period=1)

history1 = model.fit(
    crpt_data,
    orgl_data, 
    initial_epoch = 0,
    epochs = EPOCHS1,  
    batch_size = BATCH_SIZE,
    shuffle = True,     
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_input_data)
    )

np.save(path_History + "/" + "history1.npy", history1.history)

In [ ]:
history2 = model.fit(
    crpt_data, 
    orgl_data, 
    initial_epoch = EPOCHS1,
    epochs = EPOCHS1+EPOCHS2,  
    batch_size = BATCH_SIZE, 
    shuffle = True,  
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_input_data)
    )

np.save(path_History + "/" + "history2.npy", history2.history)

In [ ]:
history3 = model.fit(
    crpt_data, 
    orgl_data, 
    initial_epoch = EPOCHS1+EPOCHS2,
    epochs = EPOCHS1+2*EPOCHS2,  
    batch_size = BATCH_SIZE, 
    shuffle = True,      
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_input_data)
    )

np.save(path_History + "/" + "history3.npy", history3.history)

In [ ]:
history4 = model.fit(
    crpt_data, 
    orgl_data, 
    initial_epoch = EPOCHS1+2*EPOCHS2,
    epochs = EPOCHS1+3*EPOCHS2,   
    batch_size = BATCH_SIZE, 
    shuffle = True,       
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_input_data)
    )

np.save(path_History + "/" + "history4.npy", history4.history)

In [ ]:
history5 = model.fit(
    crpt_data, 
    orgl_data, 
    initial_epoch = EPOCHS1+3*EPOCHS2,
    epochs = EPOCHS1+4*EPOCHS2,   
    batch_size = BATCH_SIZE, 
    shuffle = True,     
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_input_data)
    )

np.save(path_History + "/" + "history5.npy", history5.history)

In [ ]:
history6 = model.fit(
    crpt_data, 
    orgl_data, 
    initial_epoch = EPOCHS1+4*EPOCHS2,
    epochs = EPOCHS1+5*EPOCHS2,   
    batch_size = BATCH_SIZE, 
    shuffle = True,      
    callbacks = [tsb, modelcp],
    validation_data=(val_input_data, val_input_data)
    )

np.save(path_History + "/" + "history6.npy", history6.history)

In [ ]:
model.load_weights(checkpoint_path)


In [ ]:
encoder = Model(inputs=[input], outputs=[bottleneck])
encoder.save_weights(encoder_path)

In [ ]:
# train image predict and display
from IPython.display import display_png

predict_no = 10
i = 0
for i in range(predict_no):    
    crpt_image = crpt_data[i]
    input_bt = np.expand_dims(crpt_image, axis=0)
    
    predict_bt = model.predict_on_batch(input_bt)

    predict = np.squeeze(predict_bt, 0)
    train_predict_image1 = np.uint8(predict*255.0 + 0.5)
    train_predict_image = Image.fromarray(train_predict_image1, mode="RGB")
    crpt_image_RGB = Image.fromarray(np.uint8(crpt_image_array_list[i]), mode="RGB")
    display_png(train_predict_image)
    display_png(crpt_image_RGB)
    print("i = ", i)
    
print("processing has finished successfully, i = ", i)

In [ ]:
# val image predict and display
from IPython.display import display_png

predict_no = 10
i = 0
for i in range(predict_no):    
    val_image = val_input_data[i]
    input_bt = np.expand_dims(val_image, axis=0)
    
    predict_bt = model.predict_on_batch(input_bt)

    predict = np.squeeze(predict_bt, 0)
    val_predict_image1 = np.uint8(predict*255.0 + 0.5)    
    val_predict_image = Image.fromarray(val_predict_image1, mode="RGB")
    val_image_RGB = Image.fromarray(np.uint8(val_image_array_list[i]), mode="RGB")
    display_png(val_predict_image)
    display_png(val_image_RGB)
    print("i = ", i)

print("processing has finished successfully, i = ", i)

In [ ]:
from IPython.display import display_png
path_train_predict_folder = "C:/Users/test/Documents/Cityscapes/Predict/Train_Predict" #path to Predict folder
predict_no = 10
i = 0
for i in range(predict_no):    
    crpt_image = crpt_data[i]
    input_bt = np.expand_dims(crpt_image, axis=0)
    
    predict_bt = model.predict_on_batch(input_bt)

    predict = np.squeeze(predict_bt, 0)
    train_predict_image1 = np.uint8(predict*255.0 + 0.5)
    train_predict_image = Image.fromarray(train_predict_image1, mode="RGB")
    crpt_image_RGB = Image.fromarray(np.uint8(crpt_image_array_list[i]), mode="RGB")
    display_png(train_predict_image)
    display_png(crpt_image_RGB)
    
    filename = "orglmage" + str(i) + ".png"
    orgl_image = Image.fromarray(np.uint8(orgl_image_array_list[i]), mode="RGB")
    orgl_image.save(path_train_predict_folder + "/" + filename)

    filename = "crptimage" + str(i) + ".png"
    crpt_image = Image.fromarray(np.uint8(crpt_image_array_list[i]), mode="RGB")
    crpt_image.save(path_train_predict_folder + "/" + filename)

    filename = "prdtimage" + str(i) + ".png"
    train_predict_image.save(path_train_predict_folder + "/" + filename)
        
    print("i = ", i)
    
print("processing has finished successfully, i = ", i)

In [ ]:
# val image predict and display
from IPython.display import display_png
path_val_predict_folder = "C:/Users/test/Documents/Cityscapes/Predict/Val_Predict" #path to Predict folder

predict_no = 10
i = 0
for i in range(predict_no):    
    val_image = val_input_data[i]
    input_bt = np.expand_dims(val_image, axis=0)
    
    predict_bt = model.predict_on_batch(input_bt)

    predict = np.squeeze(predict_bt, 0)
    val_predict_image1 = np.uint8(predict*255.0 + 0.5)    
    val_predict_image = Image.fromarray(val_predict_image1, mode="RGB")
    val_image_RGB = Image.fromarray(np.uint8(val_image_array_list[i]), mode="RGB")
    display_png(val_predict_image)
    display_png(val_image_RGB)
    
    filename = "valmage" + str(i) + ".png"
    val_image = Image.fromarray(np.uint8(val_image_array_list[i]), mode="RGB")
    val_image.save(path_val_predict_folder + "/" + filename)
    
    filename = "valprdtimage" + str(i) + ".png"
    val_predict_image.save(path_val_predict_folder + "/" + filename)    
    
    print("i = ", i)

print("processing has finished successfully, i = ", i)

In [ ]:
train_loss = model.evaluate(crpt_data, orgl_data)
print("Restored model, train_loss: {:.4f}".format(train_loss))


In [ ]:
val_loss = model.evaluate(val_input_data, val_input_data)
print("Restored model, val_loss: {:.4f}".format(val_loss))
